In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 设置请求头部信息，模拟浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
# 设置YachtWorld网站上的搜索条件
search_params = {
    'class': 'sail',
    # 'price': 'USD100000-500000',
    # 'length': '40-50ft'
}
base_url = 'https://www.yachtworld.com'

In [2]:
# 发出HTTP请求，获取搜索结果页面
# res = requests.get(f'{base_url}/boats-for-sale/', params=search_params, headers=headers)
# # print(res.content)
# soup = BeautifulSoup(res.text, 'html.parser')
pages = 1
# pages = 147
soups = [BeautifulSoup(requests.get(
    f'https://www.yachtworld.com/boats-for-sale/condition-new/type-sail/?page={i+1}', 
    headers=headers
    ).text, 'html.parser') for i in range(pages)]

In [14]:
#%%
a = soups[0].select('#root > div.search > div.flex.flex-row > div.search-right-col > div.pagination-and-results-container.search-display > div.listings-container')
a = a[0]
a

<div class="listings-container" typeof="ItemList" vocab="https://schema.org/"><meta content="2193" property="numberOfItems"/><div><div class=""><div class="mobile-leaderboard-alpha" id="div-gpt-mobile-leaderboard-1"></div></div></div><div><div class="leaderboard-alpha-wrapper"><div class="leaderboard-alpha" id="div-gpt-leaderboard-alpha"></div></div></div><a data-reporting-click-listing-type="enhanced listing" data-reporting-click-product-id="8592752" data-reporting-page="1" data-reporting-rank="1" href="https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-490-8592752/" name="&amp;lpos=P1S1&amp;lid=SR_Detail_Photo" property="itemListElement" typeof="http://schema.org/ListItem"><meta content="https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-490-8592752/" property="url"/><meta content="1" property="position"/><div class="row listing-card enhanced" id="8592752"><div class="listing-card-image-container"><meta content="//images.yachtworld.com/resize/1/36/35/6843635_201904261

In [19]:
a.findAll('a')

[<a data-reporting-click-listing-type="enhanced listing" data-reporting-click-product-id="8592752" data-reporting-page="1" data-reporting-rank="1" href="https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-490-8592752/" name="&amp;lpos=P1S1&amp;lid=SR_Detail_Photo" property="itemListElement" typeof="http://schema.org/ListItem"><meta content="https://www.yachtworld.com/yacht/2023-jeanneau-sun-odyssey-490-8592752/" property="url"/><meta content="1" property="position"/><div class="row listing-card enhanced" id="8592752"><div class="listing-card-image-container"><meta content="//images.yachtworld.com/resize/1/36/35/6843635_20190426144401065_1_LARGE.jpg?w=376&amp;h=253&amp;t=1668133949" property="image"/><img alt="Jeanneau Sun Odyssey 490" class="listing-card-image" height="253" src="//images.yachtworld.com/resize/1/36/35/6843635_20190426144401065_1_LARGE.jpg?w=376&amp;h=253&amp;t=1668133949" width="376"/><div class="banner IN_STOCK">In-Stock</div></div><div class="listing-card-infor

In [23]:
import re
# for link  in a.find_all(name='a',attrs={"href":re.compile(r'^http:')}):
# for link  in a.find_all(name='a'):
#     print(link.get('href'))
links = [link.get('href') for link  in a.find_all(name='a')]
len(links)

15

In [25]:
link = links[0]
res = requests.get(link, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
basic = soup.select('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details > div:nth-child(1) > div.collapsible.no-pad')
basic
len(basic)
basic = basic[0]

[<div class="collapsible no-pad"><div class="detail-data-table details"><table class="datatable-section"><tbody class="datatable-section"><tr class="datatable-item"><td class="datatable-title">Year</td><td class="datatable-value">2023</td></tr><tr class="datatable-item"><td class="datatable-title">Make</td><td class="datatable-value">Jeanneau</td></tr><tr class="datatable-item"><td class="datatable-title">Model</td><td class="datatable-value">Sun Odyssey 490</td></tr><tr class="datatable-item"><td class="datatable-title">Class</td><td class="datatable-value">Cruiser</td></tr><tr class="datatable-item"><td class="datatable-title">Length</td><td class="datatable-value">49ft</td></tr><tr class="datatable-item"><td class="datatable-title">Fuel Type</td><td class="datatable-value">Diesel</td></tr><tr class="datatable-item"><td class="datatable-title">Hull Material</td><td class="datatable-value">Fiberglass</td></tr><tr class="datatable-item"><td class="datatable-title">Hull Shape</td><td cl

In [33]:
# basic.find_all('td')
titles = [t.text for t in basic.select('td.datatable-title')]
titles
values = [t.text for t in basic.select('td.datatable-value')]
values

['2023',
 'Jeanneau',
 'Sun Odyssey 490',
 'Cruiser',
 '49ft',
 'Diesel',
 'Fiberglass',
 'Monohull',
 '7 years',
 'Marine Servicenter - Seattle']

In [34]:
# 爬取所有帆船的year属性和价格数据，并将它们存储到CSV文件中
def get_yacht_data(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    basic = soup.select('#BoatDetails > div.content.nav-slide.with-sticky-contact.home-block > div.boat-details > div.body > div.boat-details-content > div.details > div:nth-child(1) > div.collapsible.no-pad')
    basic = basic[0]
    titles = [t.text for t in basic.select('td.datatable-title')]
    values = [t.text for t in basic.select('td.datatable-value')]
    return {t:v for t,v in zip(titles, values)}
    
yacht_data_list = []
for link in links:
    yacht_data = get_yacht_data(link)
    yacht_data_list.append(yacht_data)
df = pd.DataFrame(yacht_data_list)
df.to_csv('yacht_data.csv', index=False)